# Pattern Recognition

In [2]:
from ml.neural_network.activations import relu, softmax
from ml.neural_network.layers import Conv2D, MaxPool2D, Flatten, FullyConnected
from ml.neural_network.losses import sparse_categorical_cross_entropy
from ml.neural_network import NeuralNetworkClassifier
from optimization.unconstrained.stochastic import Adam
from utils import load_mnist
from sklearn.metrics import classification_report
from utils import plot_model_accuracy, plot_model_loss

In [3]:
X_train, X_test, y_train, y_test = load_mnist()

cnn = NeuralNetworkClassifier(
    (Conv2D(in_channels=1, out_channels=6, kernel_size=(5, 5), strides=(1, 1),
            padding='same', channels_last=True, activation=relu),  # => [n,28,28,6]
     MaxPool2D(pool_size=(2, 2), strides=(2, 2)),  # => [n,14,14,6]
     Conv2D(in_channels=6, out_channels=16, kernel_size=(5, 5), strides=(1, 1),
            padding='same', channels_last=True, activation=relu),  # => [n,14,14,16]
     MaxPool2D(pool_size=(2, 2), strides=(2, 2)),  # => [n,7,7,16]
     Flatten(),  # => [n,7*7*16]
     FullyConnected(n_in=7 * 7 * 16, n_out=10, activation=softmax)),
    loss=sparse_categorical_cross_entropy, optimizer=Adam, learning_rate=0.002, epochs=300, batch_size=64, verbose=10)
cnn.fit(X_train, y_train)
plot_model_loss(cnn.loss_history)
plot_model_accuracy(cnn.accuracy_history)
print(classification_report(y_test, cnn.predict(X_test)))

TypeError: 'method' object is not subscriptable